In [1]:
%reload_ext autoreload
%autoreload 2

import sys
sys.path.append("../../")
sys.path.append("../")
import os
print(os.getcwd())
from models.basic import CNN
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

# width_to_output_mapping = {}
# device = "cuda"

# cnn = CNN(nc=1, cnn_type="default64", first_conv_opts=None)
# cnn.to(device)
# for width in range(233,240):
#     try:
#         t = torch.zeros(1, 1, 32, width).to(device)
#     except:
#         device = "cpu"
#         t = torch.zeros(1, 1, 32, width).to(device)

#     shape = cnn(t).shape
#     width_to_output_mapping[width] = shape[0]
    

/media/data/GitHub/simple_hwr/hwr_utils/testing


In [2]:
import tempfile
from pathlib import Path
import numpy as np
from torch.utils.data import DataLoader
from models.basic import CNN, BidirectionalRNN
from torch import nn
from loss_module.stroke_recovery_loss import StrokeLoss
import torch
from models.CoordConv import CoordConv
from trainers import TrainerStrokeRecovery
from hwr_utils.stroke_dataset import StrokeRecoveryDataset, read_img
from hwr_utils.stroke_recovery import *
from hwr_utils import utils, stroke_recovery
from torch.optim import lr_scheduler
from timeit import default_timer as timer
from train_stroke_recovery import graph
from hwr_utils.distortions import *
from hwr_utils.stroke_plotting import *

## DRAW THE IMAGE
#Pillow-simd

from PIL import Image, ImageDraw

def loader():
    folder = Path("online_coordinate_data/3_stroke_vSmall")
    folder = Path("online_coordinate_data/8_stroke_vSmall_16")
    folder = Path("online_coordinate_data/MAX_stroke_vTEST_AUGMENTFull")

    print(os.getcwd())
    x_relative_positions = True
    test_size = 500
    train_size = None
    batch_size=8
    #gt_format = ["x","y","stroke_number","eos"]
    gt_format = ["x","y","sos","eos"]

    test_dataset=StrokeRecoveryDataset([folder / "test_online_coords.json"],
                            img_height = 61,
                            num_of_channels = 1.,
                            max_images_to_load = test_size,
                            root=r"../../data",
                            cnn=None,
                            gt_format=gt_format,
                            )

    test_dataloader = DataLoader(test_dataset,
                                  batch_size=batch_size,
                                  shuffle=True,
                                  num_workers=3,
                                  collate_fn=test_dataset.collate,
                                  pin_memory=False)

    device="cuda"
    globals().update(locals())
    return test_dataloader

def reload_and_get_new_example():
    test_dataloader = loader()
    return next(iter(test_dataloader))

def get_instance():
#     if not 'test_dataloader' in globals():
#         test_dataloader = loader()
    example = next(iter(test_dataloader)) # BATCH, WIDTH, VOCAB
    instance = example["gt_list"][0].numpy()
    return instance, example

def test_drawing():
    instance, example = get_instance()
    
    draw_from_gt(instance)
    draw_from_raw(gt_to_raw(instance))

loader()
#test_drawing()

# example = next(iter(test_dataloader)) # BATCH, WIDTH, VOCAB
# instance = example["gt_list"][0].numpy()
instance, example = get_instance()

print(instance.shape)
#swap_strokes_left()

/media/data/GitHub/simple_hwr/hwr_utils/testing
../../data/online_coordinate_data/MAX_stroke_vTEST_AUGMENTFull/test_online_coords.json
Loading .npy version


INFO:root.hwr_utils.stroke_dataset:('Original dataloader size', 501)
INFO:root.hwr_utils.stroke_dataset:('Dataloader size', 500)
../../hwr_utils/stroke_dataset.py:201: UserWarning: UNK NUMBER OF SAMPLES!!!
  warnings.warn("UNK NUMBER OF SAMPLES!!!")


(544, 4)


In [16]:
# What if we only target i's?
    # Very short strokes
    # Swap over if the furtherest left stroke of next stroke is further right than the current one

def test_drawing_swaps(gt):
    print(gt.shape)
    data = draw_from_gt(gt, show=False, right_padding="random", color=(0,0,0), linewidth=1, use_stroke_number=False)[::-1]
    img = Image.fromarray(data, "RGB")
    #img.show()

    # Create an image
    swapped_strokes = Image.new("RGB", (data.shape[:2][::-1]), (255, 255, 255))
    swapped_strokes.paste(img)
    draw = ImageDraw.Draw(swapped_strokes)

    new_gt = swap_strokes_left(gt, stroke_numbers=False)

    new_gt_pil = list(gt_to_pil(new_gt*61, stroke_number=False))
    gt_pil = list(gt_to_pil(gt*61, stroke_number=False))

    #print(len(gt_pil))
    #STop
    show=False
    for i in range(len(new_gt_pil)):
        if len(gt_pil[i]) != len(new_gt_pil[i]) or not np.allclose(gt_pil[i], new_gt_pil[i]):
            draw.line(gt_pil[i], fill=(0,0,255), width=2)
            draw.line(new_gt_pil[i], fill=(0,255,0), width=2)
            draw_ellipse(gt_pil[i], draw, color=(0,0,255), width=2)
            draw_ellipse(new_gt_pil[i], draw, color=(0,255,0), width=1)

            
            show=True
            break
    if show:
        Image.fromarray((np.array(swapped_strokes)[::-1]), "RGB").show()

def draw_ellipse(point, draw, color=(0,0,255), width=2):
    line1 = [x-width/2 for x in point[0]]
    line2 = [x+width/2 for x in point[0]]
    point = np.r_[line1, line2].flatten().tolist()
    draw.ellipse(point, fill=color, outline=color)

    #overlay_images(background, instance)

In [ ]:
for i in test_dataloader:
    for ii in i["gt_list"]:
        test_drawing_swaps(ii.numpy())
    input()

(479, 4)
(845, 4)
(1174, 4)
(774, 4)
(807, 4)
(1125, 4)
(583, 4)
(626, 4)


(815, 4)
(578, 4)
(855, 4)
(277, 4)
(1121, 4)
(930, 4)
(733, 4)
(286, 4)


(784, 4)
(919, 4)
(687, 4)
(1047, 4)
(585, 4)
(840, 4)
(893, 4)
(1016, 4)


(966, 4)
(1095, 4)
(911, 4)
(1080, 4)
(477, 4)
(272, 4)
(1017, 4)
(627, 4)


(641, 4)
(171, 4)
(790, 4)
(829, 4)
(720, 4)
(552, 4)
(818, 4)
(739, 4)


...

(517, 4)
(922, 4)
(1213, 4)
(347, 4)
(727, 4)
(718, 4)
(509, 4)
(625, 4)


(239, 4)
(1060, 4)
(699, 4)
(619, 4)
(860, 4)
(610, 4)
(700, 4)
(314, 4)


(801, 4)
(1115, 4)
(771, 4)
(904, 4)
(672, 4)
(970, 4)
(808, 4)
(91, 4)


(763, 4)
(502, 4)
(662, 4)
(1174, 4)
(600, 4)
(566, 4)
(655, 4)
(525, 4)


(506, 4)
(1018, 4)
(1144, 4)
(877, 4)
(1082, 4)
(517, 4)
(619, 4)
(785, 4)
